## Domácí úkol

Za domácí úkol budete mít vyřešit pomocí optimalizace mravenčí kolonií [Vehicle Routing Problem](https://en.wikipedia.org/wiki/Vehicle_routing_problem), což je vlastně jen zobecněný problém obchodního cestujícího na princip optimalizace rozvozu zásilek doručovací společnosti. Jedná se o to, že máme depa, každé má svá vlastní vozidla s danou kapacitou a nějakou množinu zásilek, které je potřeba rozvézt k jejich majitelům. Cílem je najít množinu doručovacích tras tak, aby byly všechny zásilky dodány majitelům a aby byly minimalizované celkové náklady, tedy aby byl použit co nejmenší počet vozidel a aby byly trasy co nejkratší.

V našem případě použijeme zjednodušenou verzi tohoto problému s jedním depem, které má neomezený počet vozidel jednoho typu. Vstupní data najdete ve složce *domaci_ukol_data*, jsou ve formátu xml a obsahují 3 soubory -- 2 malé a jeden větší, které zároveň obsahují:
- Seznam uzlů se souřadnicemi x a y, kdy uzel s typem 0 je depo, a ty ostatní s typem 1 jsou lokace zákazníků.
- Seznam vozidel, v tomto případě máme jeden typ vozidla, které musí začínat a končit v depu a má nějakou maximální kapacitu předmětů, které uveze.
- Seznam požadavků, neboli do jakého uzlu se toho má co dovézt.

Svůj kód, popis řešení, výsledky a jejich rozbor mi pošlete emailem do stanoveného deadline. Pro sepsání popisu algoritmu a výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/11_rojove%20algoritmy/DU3_spojita_optimalizace.pdf).

In [1]:
from collections import namedtuple
import math
import functools
import numpy as np
import csv
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import collections as mc

In [2]:
Vertex = namedtuple('Vertex', ['id', 'x', 'y'])
Request = namedtuple('Request', ['target', 'weight'])

In [3]:
import xml.etree.ElementTree as ET

tree = ET.parse('.\\domaci_ukol_data\\data_32.xml')
root = tree.getroot()

vertices = []
requests = []

nodes = root.find('network').find('nodes')
vehicle_profile = root.find('fleet').find('vehicle_profile')
items = root.find('requests')

sdfsfd = nodes.findall('node')

for node in nodes.findall('node'):
    vertices.append(Vertex(int(node.attrib['id'])-1, float(node.find('cx').text), float(node.find('cy').text)))

for item in items.findall('request'):
    requests.append(Request(int(item.attrib['node'])-1, float(item.find('quantity').text)))

# pprint.pprint(vertices)
print(vertices)
print(requests)

[Vertex(id=0, x=82.0, y=76.0), Vertex(id=1, x=96.0, y=44.0), Vertex(id=2, x=50.0, y=5.0), Vertex(id=3, x=49.0, y=8.0), Vertex(id=4, x=13.0, y=7.0), Vertex(id=5, x=29.0, y=89.0), Vertex(id=6, x=58.0, y=30.0), Vertex(id=7, x=84.0, y=39.0), Vertex(id=8, x=14.0, y=24.0), Vertex(id=9, x=2.0, y=39.0), Vertex(id=10, x=3.0, y=82.0), Vertex(id=11, x=5.0, y=10.0), Vertex(id=12, x=98.0, y=52.0), Vertex(id=13, x=84.0, y=25.0), Vertex(id=14, x=61.0, y=59.0), Vertex(id=15, x=1.0, y=65.0), Vertex(id=16, x=88.0, y=51.0), Vertex(id=17, x=91.0, y=2.0), Vertex(id=18, x=19.0, y=32.0), Vertex(id=19, x=93.0, y=3.0), Vertex(id=20, x=50.0, y=93.0), Vertex(id=21, x=98.0, y=14.0), Vertex(id=22, x=5.0, y=42.0), Vertex(id=23, x=42.0, y=9.0), Vertex(id=24, x=61.0, y=62.0), Vertex(id=25, x=9.0, y=97.0), Vertex(id=26, x=80.0, y=55.0), Vertex(id=27, x=57.0, y=69.0), Vertex(id=28, x=23.0, y=15.0), Vertex(id=29, x=20.0, y=70.0), Vertex(id=30, x=85.0, y=60.0), Vertex(id=31, x=98.0, y=5.0)]
[Request(target=1, weight=19.0

In [4]:
def distance_matrix(vertices):
    distances = np.zeros((len(vertices), len(vertices)))
    for i in range(len(vertices)):
        v1 = vertices[i]
        for j in range(i, len(vertices)):
            v2 = vertices[j]
            dist = math.sqrt((v1.x - v2.x)**2+(v1.y - v2.y)**2)
            distances[i,j] = dist
            distances[j,i] = dist


In [5]:
def fitness(vertices, distance, solution):
    solution_distance = 0
    for x, y in zip(solution, solution[1:]):
        solution_distance += distance(vertices[x], vertices[y])
    solution_distance += distance(vertices[solution[-1]], vertices[solution[0]])
    return solution_distance

In [6]:
distances = distance_matrix(vertices)